<a href="https://colab.research.google.com/github/weieric1001/nfsepsisanalysis/blob/main/tools/Sepsis_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""預測模型效能.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/137s6oP_rnmgjWHdZdHeg5uKwPnz8twit
"""


'預測模型效能.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/137s6oP_rnmgjWHdZdHeg5uKwPnz8twit\n'

In [1]:
#呼叫套件




from google.colab import drive
import os  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pydotplus



import numpy  #數字運算
from sklearn.tree import DecisionTreeRegressor  # 決策樹
from sklearn.preprocessing import LabelEncoder  # 將string 替換成 int
from sklearn.model_selection import train_test_split # 將資料集劃分
#from sklearn.externals import joblib  #將 model 儲存下來
from sklearn.metrics import accuracy_score #測試predict 資料正確
import pandas as pd  # 讀取 csv file



from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_curve, auc

from sklearn import tree
from sklearn import datasets
import pydotplus
from sklearn.model_selection import train_test_split


import graphviz

from sklearn.tree import export_graphviz 
#from sklearn.externals.six import StringIO 
from IPython.display import Image
import pydotplus

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
#呼叫雲端硬碟

drive.mount("/content/drive")

import os

for root, dirs, files in os.walk("/content/drive/My Drive/Sepsis_Study/"):
    for filename in files:
        print(filename)

Mounted at /content/drive
Sepsis_15TB.csv


In [3]:
#呼叫雲端資料夾_檔案資料
df=pd.read_csv('/content/drive/My Drive/Sepsis_Study/Sepsis_15TB.csv')
df.head(20)

,gcs,meds_ams15b,meds_plt150b,sofa_res,sofa_ner,sofa_liver,sofa_coag,sofa_renal,bun,cre,plt,FIO2_percent,PF_ratio,fio2_per,fio2_cb,sofa_sepsis
0,15,0,3,0,0,0,2,0,32.799999,1.13,81.0,NaN,NaN,NaN,NaN,1
1,15,0,0,1,0,0,0,1,18.600000,1.20,215.0,0.33,303.030304,33.0,33.0,1
2,15,0,3,0,0,0,2,0,14.300000,1.06,93.0,0.21,476.190491,21.0,21.0,1
3,15,0,3,1,0,0,1,1,17.000000,1.42,149.0,0.33,303.030304,33.0,33.0,1
4,4,2,0,1,4,0,0,0,7.500000,0.35,239.0,0.33,303.030304,33.0,33.0,1
5,15,0,3,0,0,0,2,1,29.000000,1.66,77.0,NaN,NaN,NaN,NaN,1
6,10,2,0,3,2,0,0,2,68.000000,2.08,238.0,0.51,196.078430,51.0,51.0,1
7,15,0,3,0,0,0,1,2,23.700001,2.17,136.0,0.21,476.190491,21.0,21.0,1
8,15,0,0,1,0,0,0,0,7.900000,0.47,271.0,0.33,303.030304,33.0,33.0,0
9,15,0,0,1,0,0,0,0,10.300000,0.91,274.0,0.33,303.030304,33.0,33.0,0


In [13]:
#資料集洗牌
alldata = df.values
#np.random.shuffle(alldata)
alldata = pd.DataFrame(alldata, columns=df.columns[0: 252])
alldata

alldata.shape

(560, 16)

In [14]:
#編碼後的X與y
X = alldata.iloc[:,:15]
y = alldata.iloc[:,15]
y

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
555    1.0
556    0.0
557    1.0
558    1.0
559    1.0
Name: sofa_sepsis, Length: 560, dtype: float64

In [1]:
# LR用Grid search找最佳參數_編碼  test test test 
from sklearn.linear_model import LogisticRegression

grid=[{'penalty':['l1'], 'C':[0.001, 0.01, 0.1, 1, 10], 'solver':['liblinear']},{'penalty':['l2'], 'C':[0.001, 0.01, 0.1, 1, 10]}] 
logreg = LogisticRegression()
clf = GridSearchCV(logreg, grid, scoring='roc_auc', n_jobs=-1) #
RSKF = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=87)



acc = []
mcc = []
auroc = []
sen = []
spe = []





for train_index, test_index in RSKF.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #填補遺漏值
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(X_train)
    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)
    #正規化
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train))
    X_test = pd.DataFrame(scaler.transform(X_test))
    
    clf.fit(X_train, y_train)
    print("tuned hpyerparameters :(best parameters) ",clf.best_params_)
    
    print("Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))



    y_pred = clf.best_estimator_.predict(X_test)
    acc.append(clf.best_estimator_.score(X_test, y_test))


    
    mcc.append(matthews_corrcoef(y_test, y_pred))
    fpr, tpr, thresholds = roc_curve(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])
    auroc.append(auc(fpr, tpr))
    C_matrix = confusion_matrix(y_test, y_pred)
    sen.append(C_matrix[1,1]/(C_matrix[1,1]+C_matrix[1,0]))
    spe.append(C_matrix[0,0]/(C_matrix[0,0]+C_matrix[0,1]))

import statistics
print(statistics.mean(acc), statistics.stdev(acc))
print(statistics.mean(sen), statistics.stdev(sen))
print(statistics.mean(spe), statistics.stdev(spe))
print(statistics.mean(mcc), statistics.stdev(mcc))
print(statistics.mean(auroc), statistics.stdev(auroc))

NameError: name 'GridSearchCV' is not defined

In [ ]:
# RF用Grid search找最佳參數_編碼  test test test
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
grid={"n_estimators":[100, 250, 500, 750, 1000], "max_depth":[5, 10, 15, 20], "max_features": ['sqrt', 'log2'],"criterion":['gini', 'entropy']} 
#"n_estimators":[100, 250, 500, 750, 1000], "max_depth":[50, 100, 150, 200], max_features= “sqrt”, “log2”

clf = GridSearchCV(rf, grid, scoring='roc_auc', n_jobs=-1) #
RSKF = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=87)



acc = []
mcc = []
auroc = []
sen = []
spe = []



for train_index, test_index in RSKF.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #填補遺漏值
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(X_train)
    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)
    #正規化
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train))
    X_test = pd.DataFrame(scaler.transform(X_test))
    
    clf.fit(X_train, y_train)




    print("tuned hpyerparameters :(best parameters) ",clf.best_params_)
    
    print("Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))



    y_pred = clf.best_estimator_.predict(X_test)
    acc.append(clf.best_estimator_.score(X_test, y_test))


    
    mcc.append(matthews_corrcoef(y_test, y_pred))
    fpr, tpr, thresholds = roc_curve(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])
    auroc.append(auc(fpr, tpr))
    C_matrix = confusion_matrix(y_test, y_pred)
    sen.append(C_matrix[1,1]/(C_matrix[1,1]+C_matrix[1,0]))
    spe.append(C_matrix[0,0]/(C_matrix[0,0]+C_matrix[0,1]))

import statistics
print(statistics.mean(acc), statistics.stdev(acc))
print(statistics.mean(sen), statistics.stdev(sen))
print(statistics.mean(spe), statistics.stdev(spe))
print(statistics.mean(mcc), statistics.stdev(mcc))
print(statistics.mean(auroc), statistics.stdev(auroc))

tuned hpyerparameters :(best parameters)  {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100}
Grid scores on development set:
0.990 (+/-0.020) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 100}
0.987 (+/-0.022) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 250}
0.987 (+/-0.020) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 500}
0.988 (+/-0.020) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 750}
0.987 (+/-0.021) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 1000}
0.987 (+/-0.019) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100}
0.987 (+/-0.021) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 250}
0.987 (+/-0.023) for {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 500}
0.988 

KeyboardInterrupt: ignored

In [ ]:
#SVM調參

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

svm = SVC(probability=True, max_iter = 1000)
grid= {"kernel":['rbf','poly','sigmoid'], "C":[ 0.001, 0.01, 0.1, 1, 10 ], "gamma":[ 0.01, 0.001, 0.0001]} #'scale','auto'
clf = GridSearchCV(svm, grid, scoring='roc_auc', n_jobs=-1)



acc = []
mcc = []
auroc = []
sen = []
spe = []




RSKF = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=87)

for train_index, test_index in RSKF.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #填補遺漏值
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(X_train)
    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)
    #正規化
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train))
    X_test = pd.DataFrame(scaler.transform(X_test))
    
    clf.fit(X_train, y_train)

    print("tuned hpyerparameters :(best parameters) ",clf.best_params_)
    
    print("Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))



    y_pred = clf.best_estimator_.predict(X_test)
    acc.append(clf.best_estimator_.score(X_test, y_test))


    
    mcc.append(matthews_corrcoef(y_test, y_pred))
    fpr, tpr, thresholds = roc_curve(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])
    auroc.append(auc(fpr, tpr))
    C_matrix = confusion_matrix(y_test, y_pred)
    sen.append(C_matrix[1,1]/(C_matrix[1,1]+C_matrix[1,0]))
    spe.append(C_matrix[0,0]/(C_matrix[0,0]+C_matrix[0,1]))

#ALL
import statistics
print(statistics.mean(acc), statistics.stdev(acc))
print(statistics.mean(sen), statistics.stdev(sen))
print(statistics.mean(spe), statistics.stdev(spe))
print(statistics.mean(mcc), statistics.stdev(mcc))
print(statistics.mean(auroc), statistics.stdev(auroc))

tuned hpyerparameters :(best parameters)  {'C': 10, 'gamma': 0.01, 'kernel': 'sigmoid'}
Grid scores on development set:
0.981 (+/-0.017) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
0.953 (+/-0.047) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'poly'}
0.978 (+/-0.026) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.978 (+/-0.023) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
0.953 (+/-0.047) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'poly'}
0.977 (+/-0.024) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.977 (+/-0.024) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
0.953 (+/-0.047) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'poly'}
0.977 (+/-0.024) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.981 (+/-0.017) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.953 (+/-0.047) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
0.978 (+/-0.026) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.978 (+/-0.025) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rb

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


tuned hpyerparameters :(best parameters)  {'C': 10, 'gamma': 0.01, 'kernel': 'sigmoid'}
Grid scores on development set:
0.970 (+/-0.030) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
0.924 (+/-0.019) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'poly'}
0.969 (+/-0.030) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.968 (+/-0.028) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
0.924 (+/-0.019) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'poly'}
0.968 (+/-0.027) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.968 (+/-0.026) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
0.924 (+/-0.019) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'poly'}
0.968 (+/-0.027) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.970 (+/-0.028) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.941 (+/-0.022) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
0.968 (+/-0.031) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.969 (+/-0.031) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rb

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


tuned hpyerparameters :(best parameters)  {'C': 10, 'gamma': 0.01, 'kernel': 'sigmoid'}
Grid scores on development set:
0.975 (+/-0.017) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
0.938 (+/-0.047) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'poly'}
0.972 (+/-0.019) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.972 (+/-0.019) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
0.938 (+/-0.047) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'poly'}
0.971 (+/-0.019) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.971 (+/-0.019) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
0.938 (+/-0.047) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'poly'}
0.971 (+/-0.019) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.975 (+/-0.016) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.941 (+/-0.045) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
0.971 (+/-0.020) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.972 (+/-0.019) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rb

...

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


tuned hpyerparameters :(best parameters)  {'C': 10, 'gamma': 0.01, 'kernel': 'sigmoid'}
Grid scores on development set:
0.976 (+/-0.023) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
0.944 (+/-0.039) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'poly'}
0.974 (+/-0.026) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.974 (+/-0.024) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
0.944 (+/-0.039) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'poly'}
0.973 (+/-0.025) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.973 (+/-0.024) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
0.944 (+/-0.039) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'poly'}
0.973 (+/-0.025) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.976 (+/-0.023) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.947 (+/-0.032) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
0.974 (+/-0.026) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.975 (+/-0.024) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rb

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


tuned hpyerparameters :(best parameters)  {'C': 10, 'gamma': 0.01, 'kernel': 'sigmoid'}
Grid scores on development set:
0.976 (+/-0.029) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
0.926 (+/-0.059) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'poly'}
0.973 (+/-0.028) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.973 (+/-0.026) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
0.926 (+/-0.059) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'poly'}
0.973 (+/-0.026) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.973 (+/-0.026) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
0.926 (+/-0.059) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'poly'}
0.973 (+/-0.026) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.977 (+/-0.027) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.937 (+/-0.020) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'poly'}
0.973 (+/-0.028) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.974 (+/-0.029) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rb

KeyboardInterrupt: ignored

In [16]:
# LR
import pickle
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C= 1, penalty= 'l2')

X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#填補遺漏值
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)

#正規化
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)
print(X_train_mean)
print(X_train_std)
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

# 訓練模型3 Logistic Regression
print("Training ...")
logreg.fit(X_train, y_train)

# 評估模型3 Logistic Regression
print("Testing ...")
# 計算訓練資料集的準確度
X_train = pd.DataFrame(scaler.fit_transform(X_train))
accuracy = logreg.score(X_train, y_train)
print("Logistic Regression 訓練資料集的準確度 = {:.4f}".format(accuracy))

# 儲存模型
model_file_name = 'sepsis_logisticregression.pickle'
with open(model_file_name, 'wb') as f:
    pickle.dump(logreg, f)

[1.39486607e+01 4.10714286e-01 8.10267857e-01 9.84375000e-01
 4.64285714e-01 2.54464286e-01 4.33035714e-01 4.59821429e-01
 2.14172066e+01 1.37986487e+00 2.08257271e+02 3.97374103e-01
 2.78716203e+02 3.97374101e+01 4.09187279e+01]
[  2.43478292   0.80792472   1.33201711   1.09268124   0.99038747
   0.61053086   0.81830491   0.92976186  17.98494273   1.56299551
 100.61446722   0.12064543  62.87450867  12.06454286  13.74587038]
Training ...
Testing ...
Logistic Regression 訓練資料集的準確度 = 0.9866


In [17]:
# RF
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion= 'entropy', max_depth= 10, max_features= 'sqrt', n_estimators= 100)

X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#填補遺漏值
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)

#正規化
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)
print(X_train_mean)
print(X_train_std)
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

# 訓練模型3 Logistic Regression
print("Training ...")
rf.fit(X_train, y_train)

# 評估模型3 Logistic Regression
print("Testing ...")
# 計算訓練資料集的準確度
X_train = pd.DataFrame(scaler.fit_transform(X_train))
accuracy = rf.score(X_train, y_train)
print("Logistic Regression 訓練資料集的準確度 = {:.4f}".format(accuracy))

# 儲存模型
model_file_name = 'sepsis_randomForest.pickle'
with open(model_file_name, 'wb') as f:
    pickle.dump(rf, f)

[1.39486607e+01 4.10714286e-01 8.10267857e-01 9.84375000e-01
 4.64285714e-01 2.54464286e-01 4.33035714e-01 4.59821429e-01
 2.14172066e+01 1.37986487e+00 2.08257271e+02 3.97374103e-01
 2.78716203e+02 3.97374101e+01 4.09187279e+01]
[  2.43478292   0.80792472   1.33201711   1.09268124   0.99038747
   0.61053086   0.81830491   0.92976186  17.98494273   1.56299551
 100.61446722   0.12064543  62.87450867  12.06454286  13.74587038]
Training ...
Testing ...
Logistic Regression 訓練資料集的準確度 = 0.9933


In [18]:
#SVM調參

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

svm = SVC(C= 10, gamma= 0.01, kernel= 'sigmoid',probability=True, max_iter = 1000)

X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#填補遺漏值
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)

#正規化
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)
print(X_train_mean)
print(X_train_std)
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

# 訓練模型3 Logistic Regression
print("Training ...")
svm.fit(X_train, y_train)

# 評估模型3 Logistic Regression
print("Testing ...")
# 計算訓練資料集的準確度
X_train = pd.DataFrame(scaler.fit_transform(X_train))
accuracy = svm.score(X_train, y_train)
print("Support Vector Machine 訓練資料集的準確度 = {:.4f}".format(accuracy))

# 儲存模型
model_file_name = 'sepsis_supportVectorMachine.pickle'
with open(model_file_name, 'wb') as f:
    pickle.dump(svm, f)

[1.39486607e+01 4.10714286e-01 8.10267857e-01 9.84375000e-01
 4.64285714e-01 2.54464286e-01 4.33035714e-01 4.59821429e-01
 2.14172066e+01 1.37986487e+00 2.08257271e+02 3.97374103e-01
 2.78716203e+02 3.97374101e+01 4.09187279e+01]
[  2.43478292   0.80792472   1.33201711   1.09268124   0.99038747
   0.61053086   0.81830491   0.92976186  17.98494273   1.56299551
 100.61446722   0.12064543  62.87450867  12.06454286  13.74587038]
Training ...
Testing ...
Logistic Regression 訓練資料集的準確度 = 0.9799


In [ ]:
import pickle
import pandas as pd
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dropout

mean = [1.39508929e+01, 4.15178571e-01, 8.30357143e-01, 9.86607143e-01, 4.62053571e-01, 2.34375000e-01, 4.39732143e-01,
        4.62053571e-01, 2.18155634e+01, 1.37572072e+00, 2.06002237e+02, 3.96549298e-01, 2.79831681e+02, 3.96549296e+01, 4.08576389e+01]
std = [2.45809506, 0.81116207, 1.34222891, 1.07728126, 0.9925559,  0.56376965, 0.81609475,
       0.91289731, 17.41744609, 1.61262408, 98.28342531, 0.12664999, 62.46864266, 12.66499879, 14.30293763]


def sepsisPredict(gcs, meds_ams15b, meds_plt150b, sofa_res, sofa_ner, sofa_liver,
                 sofa_coag, sofa_renal, bun, cre, plt, FIO2_percent, PF_ratio, fio2_per, fio2_cb):
  pred = {}
  transform = [gcs, meds_ams15b, meds_plt150b, sofa_res, sofa_ner, sofa_liver,
                sofa_coag, sofa_renal, bun, cre, plt, FIO2_percent, PF_ratio, fio2_per, fio2_cb]
  for i in range(0,15):
      transform[i] = (transform[i]-mean[i]) / std[i]

  # randomForest
  model_file_name = 'sepsis_randomForest.pickle'
  with open(model_file_name, 'rb') as f:
      model2 = pickle.load(f)
      pred2 = model2.predict(np.array([transform]))
      score2 = model2.predict_proba(
          np.array([transform]))[0]
  pred['randomForest'] = pred2[0]
  if (score2[0] > 0.5):
      pred['randomForest_proba'] = np.round(score2[0]*100, 2)
  else:
      pred['randomForest_proba'] = np.round(score2[1]*100, 2)

  # logisticregression
  model_file_name = 'sepsis_logisticregression.pickle'
  with open(model_file_name, 'rb') as f:
      model3 = pickle.load(f)
      pred3 = model3.predict(np.array(
          [transform]))
      score3 = model3.predict_proba(np.array(
          [transform]))[0]
  pred['logisticregression'] = pred3[0]
  if (score3[0] > 0.5):
      pred['logisticregression_proba'] = np.round(score3[0]*100, 2)
  else:
      pred['logisticregression_proba'] = np.round(score3[1]*100, 2)

  # supportVectorMachine
  model_file_name = 'sepsis_supportVectorMachine.pickle'
  with open(model_file_name, 'rb') as f:
      model5 = pickle.load(f)
      pred5 = model5.predict(np.array(
          [transform]))
      score5 = model5.predict_proba(np.array(
          [transform]))[0]
  pred['supportVectorMachine'] = pred5[0]
  if (score5[0] > 0.5):
      pred['supportVectorMachine_proba'] = np.round(score5[0]*100, 2)
  else:
      pred['supportVectorMachine_proba'] = np.round(score5[1]*100, 2)

  return pred

sepsisPredict(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15)

{'randomForest': 1.0,
 'randomForest_proba': 100.0,
 'logisticregression': 1.0,
 'logisticregression_proba': 100.0,
 'supportVectorMachine': 1.0,
 'supportVectorMachine_proba': 100.0}